In [1]:
import os

import openeo
import requests
from requests import Request, Session

In [2]:
SENTINELHUB_BACKEND_URL = "http://127.0.0.1:5000"
SENTINELHUB_USERNAME = os.environ["SH_CLIENT_ID"]
SENTINELHUB_PASSWORD = os.environ["SH_CLIENT_SECRET"]

In [3]:
connection = openeo.connect(SENTINELHUB_BACKEND_URL)
connection.authenticate_basic(SENTINELHUB_USERNAME, SENTINELHUB_PASSWORD)

def send_authenticated_request(request):
    s = Session()
    r = s.send(connection.auth(request).prepare())  
    try:
        r.raise_for_status()
        return r.json()
    except requests.exceptions.JSONDecodeError as e:
        return r.headers
    except requests.exceptions.RequestException as e:
        print(e)
        print(e.response.text)
    

Find collections that can be ordered:

In [4]:
collections = connection.list_collections()
orderable_collections = [collection for collection in collections if collection.get("order:status") == "orderable"]
orderable_collections

[{'description': "PlanetScope is one of the satellite constellation operated by Planet. PlanetScope satellite constellation consists of more than 130 small satellites called Doves. Each Dove satellite is a CubeSat made of three cubic units and thus measures only 10 cm x 10 cm x 30 cm. The satellites are launched in groups, which constantly improves mission's characteristics such as revisit times, spatial and spectral resolutions. The constellation is constantly on and does not require an acquisition planning. PlanetScope is commercial data and has to be ordered by the user",
  'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
   'temporal': {'interval': [['2016-01-01T00:00:00Z', None]]}},
  'id': 'PLANETSCOPE',
  'license': 'various',
  'links': [],
  'order:status': 'orderable',
  'stac_version': '1.0.0'},
 {'description': 'Pleiades is a satellite constellation providing very high-resolution optical imagery and is owned by Airbus. It is now possible to purchase, order and access

In [5]:
collection_to_order = "SPOT"

Get properties we can filter our search by:

In [6]:
requests.get(f"{SENTINELHUB_BACKEND_URL}/collections/{collection_to_order}/queryables").json()

{'$id': 'https://stac-api.example.com/queryables',
 '$schema': 'https://json-schema.org/draft/2019-09/schema',
 'additionalProperties': True,
 'properties': {'expirationDate': {'properties': {'from': {'description': 'ISO-8601 time representing start of search interval, e.g. 2019-01-31T14:00:00+01:00',
     'format': 'date-time',
     'type': 'string'},
    'to': {'description': 'ISO-8601 time representing end of search interval, e.g. 2019-02-05T15:00:00+01:00.',
     'format': 'date-time',
     'type': 'string'}},
   'type': 'object'},
  'maxCloudCoverage': {'default': 100,
   'description': 'The maximum allowable cloud coverage in percent.',
   'format': 'double',
   'maximum': 100,
   'minimum': 0,
   'type': 'number'},
  'maxIncidenceAngle': {'default': 90,
   'description': 'The maximum allowable incidence angle in degrees.',
   'format': 'double',
   'maximum': 90,
   'minimum': 0,
   'type': 'number'},
  'maxSnowCoverage': {'default': 100,
   'description': 'The maximum allowable

Example search for available products:

In [7]:
payload = {
   "bbox":[4.9, 47.9, 5, 48],
   "datetime":"2022-05-20T00:00:00.000Z/2022-08-20T23:59:59.999Z",
   "collections":[collection_to_order],
   "limit":50,
   "filter":{
      "op":"and",
      "args":[
         {
            "op":"=",
            "args":[
               {
                  "property":"maxCloudCoverage"
               },
               67
            ]
         },
         {
            "op":"=",
            "args":[
               {
                  "property":"processingLevel"
               },
               "ALBUM"
            ]
         }
      ]
   }
}

available_products = send_authenticated_request(Request('POST', f"{SENTINELHUB_BACKEND_URL}/search", json=payload))
available_products

{'features': [{'assets': {},
   'geometry': {'coordinates': [[[4.627407610599472, 48.102412163813085],
      [4.633725079743262, 47.56701513374118],
      [5.438156271022335, 47.564157439121544],
      [5.443088162798822, 48.09849582876027],
      [4.627407610599472, 48.102412163813085]]],
    'type': 'Polygon'},
   'id': '784ec28c-7387-4a69-80a9-7764cd737683',
   'links': [],
   'properties': {'acquisitionDate': '2022-06-16T10:14:11.499Z',
    'acquisitionIdentifier': 'DS_SPOT7_202206161014115_GS1_GS1_GS1_GS1_E005N48_01871',
    'acquisitionStation': 'GS1',
    'activityId': 'f57310b9-2174-4474-b41f-9069c5854676',
    'archivingCenter': 'GS1',
    'azimuthAngle': 179.98432747737607,
    'cloudCover': 0,
    'constellation': 'SPOT',
    'correlationId': '1b38b1a7-31c3-4914-aefd-34015ccd6683',
    'expirationDate': '2032-06-27T11:27:06.212530813Z',
    'format': 'image/tiff',
    'geometryCentroid': {'lat': 47.833541741177775, 'lon': 5.035247886698641},
    'id': '784ec28c-7387-4a69-80a

Get list of existing orders:

In [8]:
send_authenticated_request(Request('GET', f"{SENTINELHUB_BACKEND_URL}/orders"))

{'links': {'currentToken': '0'},
 'orders': [{'costs': None,
   'id': '1f1676bd-fa76-4502-855e-066aefcf9887',
   'items': ['784ec28c-7387-4a69-80a9-7764cd737683'],
   'order:date': '2022-08-22T12:15:02.933596Z',
   'order:id': '1f1676bd-fa76-4502-855e-066aefcf9887',
   'order:status': 'orderable',
   'source_collection_id': 'SPOT',
   'target_collection_id': 'SPOT'},
  {'costs': None,
   'id': '3e664445-8cf0-400c-a81d-b3e77b4c5b1a',
   'items': ['988736cf-50c9-43e2-90bc-33c9675c2dea'],
   'order:date': '2022-08-17T08:25:43.072417Z',
   'order:id': '3e664445-8cf0-400c-a81d-b3e77b4c5b1a',
   'order:status': 'orderable',
   'source_collection_id': 'SPOT',
   'target_collection_id': 'SPOT'},
  {'costs': None,
   'id': '3fac2bd4-efa4-4911-be63-958beb5f5236',
   'items': ['988736cf-50c9-43e2-90bc-33c9675c2dea'],
   'order:date': '2022-08-22T14:11:31.048103Z',
   'order:id': '3fac2bd4-efa4-4911-be63-958beb5f5236',
   'order:status': 'orderable',
   'source_collection_id': 'SPOT',
   'target_c

Parameters that can be used in the orders are listed in collection information.

In [11]:
connection.describe_collection(collection_to_order)["orders_parameters"]

[{'description': 'The request area of interest geometry as a GeoJSON.',
  'name': 'geometry',
  'schema': {'oneOf': [{'properties': {'coordinates': {'items': {'items': {'items': {'format': 'double',
          'type': 'number'},
         'maxItems': 2,
         'minItems': 2,
         'type': 'array'},
        'type': 'array'},
       'type': 'array'},
      'type': {'enum': ['Polygon'], 'type': 'string'}},
     'type': 'object'},
    {'properties': {'coordinates': {'items': {'items': {'items': {'items': {'format': 'double',
           'type': 'number'},
          'maxItems': 2,
          'minItems': 2,
          'type': 'array'},
         'type': 'array'},
        'type': 'array'},
       'type': 'array'},
      'type': {'enum': ['MultiPolygon'], 'type': 'string'}},
     'type': 'object'}],
   'type': 'object'}}]

Prepare an order:

In [12]:
geometry = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          5.07293701171875,
          47.646279814396934
        ],
        [
          5.0736236572265625,
          47.64393786465263
        ],
        [
          5.077915191650391,
          47.643778839566814
        ],
        [
          5.0774431228637695,
          47.64507993968195
        ],
        [
          5.07293701171875,
          47.646279814396934
        ]
      ]
    ]
}

payload = {
   "source_collection_id": collection_to_order,
   "items":[
      available_products["features"][0]["id"]
   ],
   "parameters":{
      "geometry": geometry
   }
}

headers = send_authenticated_request(Request('POST', f"{SENTINELHUB_BACKEND_URL}/orders", json=payload))

In [13]:
send_authenticated_request(Request('GET', f"{SENTINELHUB_BACKEND_URL}/orders/{headers['OpenEO-Identifier']}"))

{'costs': None,
 'id': '2cd85aab-628f-4c2b-b8fd-7fc76ffecd56',
 'items': ['784ec28c-7387-4a69-80a9-7764cd737683'],
 'order:date': '2022-08-22T17:16:00.501124Z',
 'order:id': '2cd85aab-628f-4c2b-b8fd-7fc76ffecd56',
 'order:status': 'orderable',
 'source_collection_id': 'SPOT',
 'target_collection_id': 'SPOT'}

Confirm the order:

In [15]:
send_authenticated_request(Request('POST', f"{SENTINELHUB_BACKEND_URL}/orders/2cd85aab-628f-4c2b-b8fd-7fc76ffecd56"))

500 Server Error: INTERNAL SERVER ERROR for url: http://127.0.0.1:5000/orders/2cd85aab-628f-4c2b-b8fd-7fc76ffecd56
{"code":"Internal","id":null,"links":[],"message":"Server error: HTTP 409: Conflict. Response: {\"error\":{\"status\":409,\"reason\":\"Conflict\",\"message\":\"Can only confirm orders with status CREATED\",\"code\":\"COMMON_EXCEPTION\"}}"}



Download the data

In [19]:
spatial_extent = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          5.076949596405029,
          47.644361628000375
        ],
        [
          5.076867789030075,
          47.64428211616302
        ],
        [
          5.076948255300522,
          47.64424236019896
        ],
        [
          5.077004581689835,
          47.644264948818595
        ],
        [
          5.077015310525894,
          47.64431374020374
        ],
        [
          5.076949596405029,
          47.644361628000375
        ]
      ]
    ]
}

cube = connection.load_collection(
    collection_to_order,
    spatial_extent=spatial_extent,
    bands=["B1"],
)
cube.download("image.tiff")

OpenEoApiError: [500] Internal: Server error: 503 Server Error: Service Unavailable for url: https://services.sentinel-hub.com//api/v1/process